In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML
import tkinter as tk
from tkinter import filedialog

# --- ファイル選択関数 (変更なし) ---
def select_file(widget_to_update):
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    file_path = filedialog.askopenfilename(title="Select CSV file", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
    if file_path:
        widget_to_update.value = file_path
    root.destroy()

def select_save_path(widget_to_update):
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    file_path = filedialog.asksaveasfilename(title="Select output CSV path", defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
    if file_path:
        widget_to_update.value = file_path
    root.destroy()

# --- ユーザー入力フォームを作成 ---
style = {'description_width': 'initial'}

# ファイルパスと実験範囲
input_path_widget = widgets.Text(placeholder='Click "Browse..." to select a file', layout=widgets.Layout(width='80%'))
input_browse_button = widgets.Button(description="Browse...")
input_browse_button.on_click(lambda b: select_file(input_path_widget))
input_box = widgets.HBox([input_path_widget, input_browse_button])

output_path_widget = widgets.Text(placeholder='Click "Browse..." to set save path', layout=widgets.Layout(width='80%'))
output_browse_button = widgets.Button(description="Browse...")
output_browse_button.on_click(lambda b: select_save_path(output_path_widget))
output_box = widgets.HBox([output_path_widget, output_browse_button])

expno_box = widgets.HBox([widgets.IntText(description="開始EXPNO:", value=1, style=style), widgets.IntText(description="終了EXPNO:", value=40, style=style)])

# --- GDPシグナル用のウィジェット ---
gdp_ppm_box = widgets.VBox([
    widgets.FloatText(description="GDPシグナルの水素 (¹H) 化学シフト値 (f1_ppm列):", value=0.1, style=style, layout=widgets.Layout(width='90%')),
    widgets.FloatText(description="GDPシグナルの窒素 (¹⁵N) 化学シフト値 (f2_ppm列):", value=8.4, style=style, layout=widgets.Layout(width='90%')),
    widgets.FloatText(description="GDP探索半径 (ppm):", value=0.2, style=style, layout=widgets.Layout(width='90%'))
])

# --- GTPシグナル用のウィジェット (追加) ---
gtp_ppm_box = widgets.VBox([
    widgets.FloatText(description="GTPシグナルの水素 (¹H) 化学シフト値 (f1_ppm列):", value=0.6, style=style, layout=widgets.Layout(width='90%')),
    widgets.FloatText(description="GTPシグナルの窒素 (¹⁵N) 化学シフト値 (f2_ppm列):", value=8.7, style=style, layout=widgets.Layout(width='90%')),
    widgets.FloatText(description="GTP探索半径 (ppm):", value=0.2, style=style, layout=widgets.Layout(width='90%'))
])

# ウィジェットをまとめて表示
display(HTML("<h3>1. ファイルパスと実験範囲の設定</h3>"))
display(widgets.Label("入力CSVファイル:"))
display(input_box)
display(widgets.Label("出力CSVファイル:"))
display(output_box)
display(expno_box)

display(HTML("<h3>2. 探索するシグナルの設定</h3>"))
display(HTML("<h4>GDP-bound Signal</h4>"))
display(gdp_ppm_box)
display(HTML("<h4>GTP-bound Signal</h4>"))
display(gtp_ppm_box)

# ウィジェットへのアクセス用
start_expno_widget = expno_box.children[0]
end_expno_widget = expno_box.children[1]
gdp_f1_H_input_widget = gdp_ppm_box.children[0]
gdp_f2_N_input_widget = gdp_ppm_box.children[1]
gdp_radius_input_widget = gdp_ppm_box.children[2]
gtp_f1_H_input_widget = gtp_ppm_box.children[0]
gtp_f2_N_input_widget = gtp_ppm_box.children[1]
gtp_radius_input_widget = gtp_ppm_box.children[2]

Label(value='入力CSVファイル:')

Label(value='出力CSVファイル:')

In [2]:
import pandas as pd
import numpy as np

# --- ピーク探索用の補助関数 ---
def find_closest_peak(df, target_f1_ppm, target_f2_ppm, search_radius):
    """DataFrameの中からターゲット座標に最も近いピークを探し、その行を返す"""
    if df.empty:
        return None
        
    distances = np.sqrt(
        (df['f1_ppm'] - target_f1_ppm)**2 + 
        (df['f2_ppm'] - target_f2_ppm)**2
    )
    min_dist_idx = distances.idxmin()
    
    if distances[min_dist_idx] <= search_radius:
        return df.loc[min_dist_idx]
    else:
        return None

# --- メインの処理関数 ---
def process_data():
    # ウィジェットから値を取得
    input_csv_path = input_path_widget.value
    output_csv_path = output_path_widget.value
    start_expno = start_expno_widget.value
    end_expno = end_expno_widget.value
    
    # GDPターゲット値
    target_gdp_f1_H = gdp_f1_H_input_widget.value
    target_gdp_f2_N = gdp_f2_N_input_widget.value
    gdp_search_radius = gdp_radius_input_widget.value
    
    # GTPターゲット値
    target_gtp_f1_H = gtp_f1_H_input_widget.value
    target_gtp_f2_N = gtp_f2_N_input_widget.value
    gtp_search_radius = gtp_radius_input_widget.value

    if not input_csv_path or not output_csv_path:
        print("エラー: 入力および出力ファイルのパスを両方入力してください。")
        return

    print(f"INFO: CSVファイルを読み込んでいます: {input_csv_path}")
    try:
        df = pd.read_csv(input_csv_path)
    except FileNotFoundError:
        print(f"エラー: 入力ファイルが見つかりません。パスを確認してください: {input_csv_path}")
        return

    print("INFO: GDPおよびGTPシグナルの探索を開始します。")
    
    results = []

    # 各実験番号ごとに両方のピークを探す
    for expno in range(start_expno, end_expno + 1):
        exp_df = df[df['expno'] == expno]
        
        # GDPピークを探す
        gdp_peak = find_closest_peak(exp_df, target_gdp_f1_H, target_gdp_f2_N, gdp_search_radius)
        # GTPピークを探す
        gtp_peak = find_closest_peak(exp_df, target_gtp_f1_H, target_gtp_f2_N, gtp_search_radius)
        
        # 結果を辞書として保存
        result_row = {'expno': expno}
        result_row['gdp_intensity'] = gdp_peak['intensity'] if gdp_peak is not None else np.nan
        result_row['gtp_intensity'] = gtp_peak['intensity'] if gtp_peak is not None else np.nan
        results.append(result_row)

    # 結果をDataFrameに変換
    final_df = pd.DataFrame(results).set_index('expno')

    # --- 欠損値の補完 (reindexは不要になりましたが、念のため) ---
    full_expno_range = range(start_expno, end_expno + 1)
    final_df = final_df.reindex(full_expno_range)

    # --- 新しいCSVファイルに出力 ---
    final_df.to_csv(output_csv_path)
    print(f"完了: 処理結果を {output_csv_path} に保存しました。")
    
# 関数を実行
process_data()

エラー: 入力および出力ファイルのパスを両方入力してください。


In [ ]:
## --- ファイル選択関数 (変更なし) ---
def select_file(widget_to_update):
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    file_path = filedialog.askopenfilename(title="Select CSV file", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
    if file_path:
        widget_to_update.value = file_path
    root.destroy()

def select_save_path(widget_to_update):
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    file_path = filedialog.asksaveasfilename(title="Select output CSV path", defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
    if file_path:
        widget_to_update.value = file_path
    root.destroy()

# --- ユーザー入力フォームを作成 ---
style = {'description_width': 'initial'}

# ファイルパスと実験範囲
input_path_widget = widgets.Text(placeholder='Click "Browse..." to select a file', layout=widgets.Layout(width='80%'))
input_browse_button = widgets.Button(description="Browse...")
input_browse_button.on_click(lambda b: select_file(input_path_widget))
input_box = widgets.HBox([input_path_widget, input_browse_button])

output_path_widget = widgets.Text(placeholder='Click "Browse..." to set save path', layout=widgets.Layout(width='80%'))
output_browse_button = widgets.Button(description="Browse...")
output_browse_button.on_click(lambda b: select_save_path(output_path_widget))
output_box = widgets.HBox([output_path_widget, output_browse_button])

# 測定間隔の入力ウィジェットを追加
expno_box = widgets.HBox([
    widgets.IntText(description="開始EXPNO:", value=2, style=style),
    widgets.IntText(description="終了EXPNO:", value=41, style=style),
    widgets.IntText(description="測定間隔 (分):", value=10, style=style) # デフォルト10分
])

# --- GDPシグナル用のウィジェット ---
gdp_ppm_box = widgets.VBox([
    widgets.FloatText(description="GDPシグナルの水素 (¹H) 化学シフト値 (f1_ppm列):", value=0.7, style=style, layout=widgets.Layout(width='90%')),
    widgets.FloatText(description="GDPシグナルの窒素 (¹⁵N) 化学シフト値 (f2_ppm列):", value=9.8, style=style, layout=widgets.Layout(width='90%')),
    widgets.FloatText(description="GDP探索半径 (ppm):", value=0.1, style=style, layout=widgets.Layout(width='90%'))
])

# --- GTPシグナル用のウィジェット (追加) ---
gtp_ppm_box = widgets.VBox([
    widgets.FloatText(description="GTPシグナルの水素 (¹H) 化学シフト値 (f1_ppm列):", value=0.85, style=style, layout=widgets.Layout(width='90%')),
    widgets.FloatText(description="GTPシグナルの窒素 (¹⁵N) 化学シフト値 (f2_ppm列):", value=10.3, style=style, layout=widgets.Layout(width='90%')),
    widgets.FloatText(description="GTP探索半径 (ppm):", value=0.1, style=style, layout=widgets.Layout(width='90%'))
])

# ウィジェットをまとめて表示
display(HTML("<h3>1. ファイルパスと実験範囲の設定</h3>"))
display(widgets.Label("入力CSVファイル:"))
display(input_box)
display(widgets.Label("出力CSVファイル:"))
display(output_box)
display(expno_box)
display(HTML("<h3>2. 探索するシグナルの設定</h3>"))
display(HTML("<h4>GDP-bound Signal</h4>"))
display(gdp_ppm_box)
display(HTML("<h4>GTP-bound Signal</h4>"))
display(gtp_ppm_box)

# ウィジェットへのアクセス用
start_expno_widget = expno_box.children[0]
end_expno_widget = expno_box.children[1]
time_interval_widget = expno_box.children[2]
gdp_f1_H_input_widget = gdp_ppm_box.children[0]
gdp_f2_N_input_widget = gdp_ppm_box.children[1]
gdp_radius_input_widget = gdp_ppm_box.children[2]
gtp_f1_H_input_widget = gtp_ppm_box.children[0]
gtp_f2_N_input_widget = gtp_ppm_box.children[1]
gtp_radius_input_widget = gtp_ppm_box.children[2]